In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense
import re

In [ ]:
df = pd.read_csv('/content/Political_Data_1.csv', sep=',', encoding='utf-8')
df.head()

,Jenis_Konflik,Mediasi,Final_Deskripsi_Summary
0,Konflik Kepengurusan Partai,Musyawarah,Ketua Umum Partai Demokrat Anas Urbaningrum me...
1,Konflik Kepengurusan Partai,Musyawarah,Sudah beberapa pekan Partai Demokrat diguncang...
2,Konflik Kepengurusan Partai,Musyawarah,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...
3,Konflik Kepengurusan Partai,Musyawarah,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...
4,Konflik Kepengurusan Partai,Musyawarah,Deputi Badan Pemenangan Pemilu (Bappilu) DPP K...


In [ ]:
encoded_dict = {'Konflik Kepengurusan Partai':0, 'Konflik Dukungan Capres':1, 'Konflik Elite Politik':2, 'Konflik Caleg':3}
keys = list(encoded_dict)
df['Jenis_Konflik'] = df.Jenis_Konflik.map(encoded_dict)
df.head()

,Jenis_Konflik,Mediasi,Final_Deskripsi_Summary
0,0,Musyawarah,Ketua Umum Partai Demokrat Anas Urbaningrum me...
1,0,Musyawarah,Sudah beberapa pekan Partai Demokrat diguncang...
2,0,Musyawarah,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...
3,0,Musyawarah,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...
4,0,Musyawarah,Deputi Badan Pemenangan Pemilu (Bappilu) DPP K...


In [ ]:
encoded_dict = {'Tidak Ada':0, 'Keputusan Pimpinan':1, 'Jalur Hukum':2, 'Musyawarah':3}
keys = list(encoded_dict)
df['Mediasi'] = df.Mediasi.map(encoded_dict)
df.head()

,Jenis_Konflik,Mediasi,Final_Deskripsi_Summary
0,0,3,Ketua Umum Partai Demokrat Anas Urbaningrum me...
1,0,3,Sudah beberapa pekan Partai Demokrat diguncang...
2,0,3,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...
3,0,3,Mantan Ketua Umum DPP Partai Demokrat Anas Urb...
4,0,3,Deputi Badan Pemenangan Pemilu (Bappilu) DPP K...


In [ ]:
data = df.pop('Final_Deskripsi_Summary')
label_0 = df.pop('Jenis_Konflik')
label_1 = df.pop('Mediasi')

In [ ]:
y_label_0 = pd.get_dummies(label_0, dtype ='float32').values
print('Shape of y_label_0 (Jenis_Konflik) tensor:', y_label_0.shape)
y_label_1 = pd.get_dummies(label_1, dtype ='float32').values
print('Shape of y_label_1 (Mediasi) tensor:', y_label_1.shape)

Shape of y_label_0 (Jenis_Konflik) tensor: (1105, 4)
Shape of y_label_1 (Mediasi) tensor: (1105, 4)


In [ ]:
def data_split():
  data_train, data_test, y_train_0, y_test_0, y_train_1, y_test_1 = train_test_split(data, y_label_0, y_label_1, test_size = 0.2)
  return data_train, data_test, y_train_0, y_test_0, y_train_1, y_test_1

In [ ]:
! pip install -q transformers
import transformers
from transformers import AutoTokenizer, TFBertModel, BertConfig
from transformers import RobertaTokenizer, TFRobertaModel
from transformers import DistilBertTokenizer, TFDistilBertModel
import time

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
dim = 128
n_epoch = 30
k_fold = 5
def init_bert(data_train, data_test):
  # tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
  # bert = TFBertModel.from_pretrained('bert-base-uncased')
  tokenizer = AutoTokenizer.from_pretrained('cahya/bert-base-indonesian-522M')
  bert = TFBertModel.from_pretrained('cahya/bert-base-indonesian-522M', from_pt=True)
  # tokenizer = RobertaTokenizer.from_pretrained('cahya/roberta-base-indonesian-522M')
  # bert = TFRobertaModel.from_pretrained('cahya/roberta-base-indonesian-522M', from_pt=True)
  # tokenizer = DistilBertTokenizer.from_pretrained('cahya/distilbert-base-indonesian')
  # bert = TFDistilBertModel.from_pretrained('cahya/distilbert-base-indonesian', from_pt=True)
  x_train = tokenizer(
    text=list(data_train),
    add_special_tokens=True,
    max_length=dim,
    truncation=True,
    padding="max_length",
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
  x_test = tokenizer(
    text=list(data_test),
    add_special_tokens=True,
    max_length=dim,
    truncation=True,
    padding="max_length",
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
  return bert, x_train, x_test

In [ ]:
def customize_bert(bert):
  input_ids = keras.Input(shape = (dim,), dtype=tf.int32, name = 'input_ids')
  input_mask = keras.Input(shape = (dim,), dtype=tf.int32, name = 'attention_mask')
  embeddings = bert(input_ids, attention_mask=input_mask)[0]
  out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
  out = Dense(64, activation='relu')(out)
  out = Dense(32, activation = 'relu')(out)
  y = Dense(4, activation = 'softmax')(out)
  model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
  model.layers[2].trainable = True
  return model

In [ ]:
def customize_bert_v1(bert):
  input_ids = keras.Input(shape = (dim,), dtype=tf.int32, name = 'input_ids')
  input_mask = keras.Input(shape = (dim,), dtype=tf.int32, name = 'attention_mask')
  embeddings = bert(input_ids, attention_mask=input_mask)[0]
  # out = tf.keras.layers.Conv1D(filters=64, kernel_size=(5), padding='same',activation='relu')(embeddings)
  # out = tf.keras.layers.GlobalMaxPool1D()(out)
  out = tf.keras.layers.Reshape((dim, embeddings.shape[-1], 1))(embeddings)
  out = tf.keras.layers.Conv2D(filters=8, kernel_size=(5), padding='same',activation='relu')(out)
  out = tf.keras.layers.MaxPool2D(pool_size=(3, 6))(out)
  out = tf.keras.layers.Conv2D(filters=12, kernel_size=(5), padding='same',activation='relu')(out)
  out = tf.keras.layers.MaxPool2D(pool_size=(3, 6))(out)
  out = tf.keras.layers.Conv2D(filters=16, kernel_size=(5), padding='same',activation='relu')(out)
  out = tf.keras.layers.MaxPool2D(pool_size=(3, 6))(out)
  out = tf.keras.layers.Flatten()(out)
  out = Dense(32, activation = 'relu')(out)
  y = Dense(4, activation = 'softmax')(out)
  model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
  model.layers[2].trainable = True
  return model

In [ ]:
def customize_bert_v2(bert):
  input_ids = keras.Input(shape = (dim,), dtype=tf.int32, name = 'input_ids')
  input_mask = keras.Input(shape = (dim,), dtype=tf.int32, name = 'attention_mask')
  embeddings = bert(input_ids, attention_mask=input_mask)[0]
  out = tf.keras.layers.Reshape((dim, embeddings.shape[-1], 1))(embeddings)
  out = tf.keras.layers.Conv2D(filters=8, kernel_size=(5), padding='same',activation='relu')(out)
  out = tf.keras.layers.MaxPool2D(pool_size=(3, 6))(out)
  out = tf.keras.layers.Conv2D(filters=12, kernel_size=(5), padding='same',activation='relu')(out)
  out = tf.keras.layers.MaxPool2D(pool_size=(3, 6))(out)
  out = tf.keras.layers.Conv2D(filters=16, kernel_size=(5), padding='same',activation='relu')(out)
  out = tf.keras.layers.MaxPool2D(pool_size=(3, 6))(out)
  out = tf.keras.layers.Flatten()(out)
  out = Dense(32, activation = 'relu')(out)
  # Additional single input neuron
  single_input = keras.Input(shape=(4,), name='single_input')
  # Concatenate the additional input with the existing network
  concat_out = tf.keras.layers.Concatenate()([out, single_input])
  # Final output layer with softmax activation
  y = Dense(16, activation='relu')(concat_out)
  # Final output layer with softmax activation
  y = Dense(4, activation='softmax')(y)
  # Define the model with multiple inputs
  model = tf.keras.Model(inputs=[input_ids, input_mask, single_input], outputs=y)
  model.layers[2].trainable = True
  return model


In [ ]:
def compile_bert(model):
  optimizer = tf.keras.optimizers.Adam(
      learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website
      epsilon=1e-08,
      clipnorm=1.0
  )
  # Set loss and metrics
  loss = CategoricalCrossentropy(from_logits = True)
  metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall'),
             tf.keras.metrics.F1Score(name='f1_score', average='macro'),
             tf.keras.metrics.AUC(name='auc')]
  # Compile the model
  model.compile(optimizer = optimizer, loss = loss, metrics = metrics)
  return model

In [ ]:
temp_acc, temp_prc, temp_rec, temp_f1s, temp_auc, pred, training_time = [], [], [], [], [], [], []
for i in range (k_fold):
  data_train, data_test, y_train_0, y_test_0, y_train_1, y_test_1 = data_split()
  model, x_train, x_test = init_bert(data_train, data_test)
  model = customize_bert_v1(model)
  model = compile_bert(model)
  t0 = time.time()
  train_history = model.fit(
    x = [x_train['input_ids'],x_train['attention_mask']] ,
    y = y_train_1,
    validation_data = (
    [x_test['input_ids'], x_test['attention_mask']], y_test_1,
    ),
  epochs=30,
  batch_size=32
  )
  training_time.append(time.time() - t0)
  temp_acc.append(train_history.history['val_accuracy'])
  temp_prc.append(train_history.history['val_precision'])
  temp_rec.append(train_history.history['val_recall'])
  temp_f1s.append(train_history.history['val_f1_score'])
  temp_auc.append(train_history.history['val_auc'])
  prediction = model.predict([x_test['input_ids'], x_test['attention_mask']])
  pred.append(prediction.argmax(axis=1))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


28/28 [==============================] - 50s 352ms/step - loss: 1.1672 - accuracy: 0.3937 - precision: 0.3652 - recall: 0.1302 - f1_score: 0.2207 - auc: 0.6922 - val_loss: 1.1379 - val_accuracy: 0.3348 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.1721 - val_auc: 0.7209
Epoch 2/30
28/28 [==============================] - 4s 159ms/step - loss: 1.0674 - accuracy: 0.4400 - precision: 0.5633 - recall: 0.2165 - f1_score: 0.2445 - auc: 0.7526 - val_loss: 0.9520 - val_accuracy: 0.5973 - val_precision: 0.7402 - val_recall: 0.4455 - val_f1_score: 0.3204 - val_auc: 0.8363
Epoch 3/30
28/28 [==============================] - 4s 154ms/step - loss: 0.7775 - accuracy: 0.6505 - precision: 0.7578 - recall: 0.5937 - f1_score: 0.4800 - auc: 0.8785 - val_loss: 0.7699 - val_accuracy: 0.6425 - val_precision: 0.7000 - val_recall: 0.5972 - val_f1_score: 0.5145 - val_auc: 0.8855
Epoch 4/30
28/28 [==============================] - 4s 151ms/step - loss: 0.5863 - accuracy: 0.7115 - precis

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can alr

...

Epoch 1/30


28/28 [==============================] - 50s 346ms/step - loss: 1.3934 - accuracy: 0.3326 - precision: 0.2245 - recall: 0.0798 - f1_score: 0.1760 - auc: 0.5466 - val_loss: 1.1613 - val_accuracy: 0.4570 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.1568 - val_auc: 0.7280
Epoch 2/30
28/28 [==============================] - 4s 152ms/step - loss: 1.1246 - accuracy: 0.4423 - precision: 0.3883 - recall: 0.0484 - f1_score: 0.1667 - auc: 0.7182 - val_loss: 1.1433 - val_accuracy: 0.4570 - val_precision: 0.5635 - val_recall: 0.3447 - val_f1_score: 0.1568 - val_auc: 0.7358
Epoch 3/30
28/28 [==============================] - 4s 152ms/step - loss: 1.1148 - accuracy: 0.4514 - precision: 0.4481 - recall: 0.1149 - f1_score: 0.1983 - auc: 0.7237 - val_loss: 1.1527 - val_accuracy: 0.5158 - val_precision: 0.5000 - val_recall: 0.0340 - val_f1_score: 0.2879 - val_auc: 0.7495
Epoch 4/30
28/28 [==============================] - 4s 154ms/step - loss: 1.0856 - accuracy: 0.4683 - precis

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can alr

Epoch 1/30


28/28 [==============================] - 47s 287ms/step - loss: 1.1452 - accuracy: 0.4050 - precision: 0.5949 - recall: 0.0572 - f1_score: 0.2113 - auc: 0.7035 - val_loss: 1.1120 - val_accuracy: 0.4615 - val_precision: 0.5028 - val_recall: 0.4265 - val_f1_score: 0.1579 - val_auc: 0.7505
Epoch 2/30
28/28 [==============================] - 5s 168ms/step - loss: 1.0867 - accuracy: 0.4672 - precision: 0.6550 - recall: 0.1825 - f1_score: 0.2119 - auc: 0.7512 - val_loss: 1.0891 - val_accuracy: 0.4842 - val_precision: 0.6667 - val_recall: 0.0190 - val_f1_score: 0.2700 - val_auc: 0.7837
Epoch 3/30
28/28 [==============================] - 4s 155ms/step - loss: 0.9767 - accuracy: 0.5679 - precision: 0.7257 - recall: 0.3637 - f1_score: 0.3202 - auc: 0.8104 - val_loss: 0.9049 - val_accuracy: 0.6380 - val_precision: 0.7759 - val_recall: 0.4265 - val_f1_score: 0.3539 - val_auc: 0.8660
Epoch 4/30
28/28 [==============================] - 4s 153ms/step - loss: 0.7560 - accuracy: 0.6584 - precision: 0.7

In [ ]:
temp_acc, temp_prc, temp_rec, temp_f1s, temp_auc, pred, training_time = [], [], [], [], [], [], []
for i in range (k_fold):
  data_train, data_test, y_train_0, y_test_0, y_train_1, y_test_1 = data_split()
  model, x_train, x_test = init_bert(data_train, data_test)
  model = customize_bert_v2(model)
  model = compile_bert(model)
  t0 = time.time()
  train_history = model.fit(
    x = [x_train['input_ids'],x_train['attention_mask'], y_train_0] ,
    y = y_train_1,
    validation_data = (
    [x_test['input_ids'], x_test['attention_mask'], y_test_0], y_test_1,
    ),
  epochs=n_epoch,
  batch_size=32
  )
  training_time.append(time.time() - t0)
  temp_acc.append(train_history.history['val_accuracy'])
  temp_prc.append(train_history.history['val_precision'])
  temp_rec.append(train_history.history['val_recall'])
  temp_f1s.append(train_history.history['val_f1_score'])
  temp_auc.append(train_history.history['val_auc'])
  prediction = model.predict([x_test['input_ids'], x_test['attention_mask'], y_test_0])
  pred.append(prediction.argmax(axis=1))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Epoch 1/30


28/28 [==============================] - 52s 475ms/step - loss: 1.4338 - accuracy: 0.2760 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.2114 - auc: 0.4896 - val_loss: 1.3517 - val_accuracy: 0.2805 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.1519 - val_auc: 0.5874
Epoch 2/30
28/28 [==============================] - 9s 321ms/step - loss: 1.3205 - accuracy: 0.3801 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.3108 - auc: 0.6623 - val_loss: 1.3096 - val_accuracy: 0.3801 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.2326 - val_auc: 0.6773
Epoch 3/30
28/28 [==============================] - 9s 322ms/step - loss: 1.2970 - accuracy: 0.4423 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.3125 - auc: 0.7038 - val_loss: 1.2911 - val_accuracy: 0.5249 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.2950 - val_auc: 0.7074
Epoch 4/30
28/28 [==============================] - 9s 319ms/step - 

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can alr

Epoch 1/30


28/28 [==============================] - 53s 447ms/step - loss: 1.2805 - accuracy: 0.4502 - precision: 0.6923 - recall: 0.0204 - f1_score: 0.1633 - auc: 0.6732 - val_loss: 1.2955 - val_accuracy: 0.4027 - val_precision: 0.6429 - val_recall: 0.0407 - val_f1_score: 0.1435 - val_auc: 0.6573
Epoch 2/30
28/28 [==============================] - 9s 319ms/step - loss: 1.2330 - accuracy: 0.4536 - precision: 0.7055 - recall: 0.1301 - f1_score: 0.1560 - auc: 0.7134 - val_loss: 1.2619 - val_accuracy: 0.4072 - val_precision: 0.5194 - val_recall: 0.3032 - val_f1_score: 0.1510 - val_auc: 0.7054
Epoch 3/30
28/28 [==============================] - 9s 322ms/step - loss: 1.1164 - accuracy: 0.5577 - precision: 0.6719 - recall: 0.3360 - f1_score: 0.3219 - auc: 0.7783 - val_loss: 1.1188 - val_accuracy: 0.5656 - val_precision: 0.6698 - val_recall: 0.3213 - val_f1_score: 0.3768 - val_auc: 0.7780
Epoch 4/30
28/28 [==============================] - 9s 321ms/step - loss: 1.0166 - accuracy: 0.6029 - precision: 0.7

In [ ]:
f = open("Accuracy_BERTIndo-B_Mediasi_1.csv", "w")
f.write("epoch = {}, training_time = {}\n".format(n_epoch, sum(training_time)/len(training_time)))
f.write("iteration, accuracy, precision, recall, f1_score, auc\n")
for i in range(len(temp_acc)):
  for j in range(len(temp_acc[0])):
    f.write("{}, {}, {}, {}, {}, {}\n".format(j+1, temp_acc[i][j], temp_prc[i][j], temp_rec[i][j], temp_f1s[i][j], temp_auc[i][j]))
f.close()


In [ ]:
import matplotlib.pyplot as plt
plt.title('Accuracy')
plt.plot(train_history.history['accuracy'], label='Train')
plt.plot(train_history.history['val_accuracy'], label='Test')
plt.ylim(0, 1)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# # generate classification report
pred = model.predict( {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
pred = pred.argmax(axis=1)

print(classification_report(y_test.argmax(axis=1), pred, target_names=keys))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test.argmax(axis=1), pred))